# Data Story Project - HDI & climate

Lex Bolt: 13335022

Coen de Graaf: 13041568 

Lucas Groot: 14638274

In [2]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

The Human Development Index (HDI) is a comprehensive measure that assesses the level of development in a country. It incorporates multiple dimensions, including health, knowledge, and standard of living.

The HDI combines various indicators to provide a holistic view of development. It considers factors such as life expectancy at birth, mean years of schooling, expected years of schooling, and Gross Domestic Product (GDP) in terms of US dollars. By establishing minimum and maximum benchmarks for each dimension, known as "goalposts," the HDI enables a comparative analysis of countries' progress.

The HDI is expressed as a value between 0 and 1, with higher values indicating higher levels of human development. Its purpose is to draw the attention of policymakers, media, and non-governmental organizations, steering the focus away from purely economic statistics and towards human outcomes. It underscores the notion that people's well-being and capabilities should be the ultimate yardstick for assessing a country's development, rather than solely relying on economic growth.

This datastory will investigate the possible correlations between a country's HDI and its impact on the world's environment. By comparing a country's HDI to its CO2 emissions and reviewing the changes in climate (percipation, wind speed and sea level rise) of said country, correlations between HDI and climate change can be investigated.


https://www.who.int/data/nutrition/nlis/info/human-development-index

In [3]:
dataset = pd.read_csv('dataset_with_tonspercapita.csv')

In [12]:
column_names = ["Human Development Index (HDI) ", "Life expectancy at birth",
                "Expected years of schooling", "Gross national income (GNI) per capita"]

fig = make_subplots(rows=1, cols=1)
fig.update_layout(height=600)

dropdown = go.layout.Updatemenu(
    buttons=[
        {'label': column, 'method': 'update', 'args': [{'z': [dataset[column]]}]} for column in column_names
    ],
    direction='down',
    active=0,
    showactive=True,
    xanchor="auto"
)

fig.update_layout(
    updatemenus=[dropdown],
    title=f"<b>Components of the Human Development Index per country</b><br>Use the drop-down menu to select a component",
    title_font_color='#525252',
    title_font_size=26,
    font=dict(family='Heebo', size=18, color='#525252'),
)

fig.add_trace(
    go.Choropleth(
        locations=dataset['ISO-code'],
        z=dataset[column_names[0]],
        text=dataset['Country'],
        colorscale='inferno',
        autocolorscale=False,
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        zauto=True
    )
)

fig.update_traces(hovertemplate='Country: %{text}<br>%{z}<extra></extra>')
fig.show()

This first figure shows the Human Development Index per country of the world. As visible in the figure, countries in Western Europe, North America and Oceania have a very high HDI, whereas countries in Africa have a lower HDI. However, the dataset does not contain information about some countries. These are white in the figure.

In [27]:
fig = px.bar(dataset, x='Country', y='TonsPerCapita')
desired_order = dataset.sort_values('Human Development Index (HDI) ')['Country'].to_list()
fig.update_xaxes(categoryorder='array', categoryarray=desired_order, title="Tonnes per capita per country, countries are sorted on HDI, from low to high")
fig.update_yaxes(title="CO2 Emissions in tons per capita")
fig.show()

The plot above displays countries and their emissions represented in tons per capita, the countries are sorted on HDI, ascending from left to right. In the plot it can be seen that the emissions increase towards the right end of the chart, where the countries with a higer HDI sit. From this figure a conclusion can be drawn; the higher the HDI of a country is, the higher its emissions per capita. This conclusion is backed up by the following article from the IMF: https://www.imf.org/en/Publications/fandd/issues/2021/09/climate-change-and-inequality-guivarch-mejean-taconet

In [26]:
dataset['Human Development Index (HDI) '] = pd.to_numeric(dataset['Human Development Index (HDI) '], errors='coerce')

dataset.sort_values('Human Development Index (HDI) ', ascending=False, inplace=True)

fig = px.scatter(dataset, x="Human Development Index (HDI) ", y="TonsPerCapita", trendline="lowess", trendline_color_override="black")
fig.update_yaxes(title="CO2 Emissions in tons per capita")
fig.show()

There is a correlation between tons of CO2 per capita and HDI, but of course correlation does not imply causation. It could be that high tons of CO2 per capita and a high HDI are caused by a large Gross National Income (GNI).

One factor to consider is the energy mix used by a country. Countries with a higher GNI might rely heavily on industries that produce significant greenhouse gas emissions, such as manufacturing or fossil fuel extraction. This can result in higher CO2 emissions per capita. However, it's worth noting that countries with high GNI can also invest in cleaner technologies and renewable energy sources, which can mitigate their carbon footprint.

Moreover, countries with a higher GNI typically have more financial resources available per person. These countries can allocate greater investments towards the overall development of their nation, including improvements in healthcare, education, infrastructure, and other factors that contribute to a higher HDI.

It is important to recognize that the relationship between CO2 emissions per capita, HDI, and GNI is complex and multifaceted. Other factors, such as environmental policies, technological advancements, social priorities, and cultural norms, also influence this relationship. Therefore, a comprehensive understanding requires a careful analysis of these various factors to uncover the underlying dynamics and identify effective strategies for sustainable development.

improve this text - ChatGPT

In [10]:
dataset['Gross national income (GNI) per capita'].replace('..', '1', inplace=True)
dataset['Gross national income (GNI) per capita'] = pd.to_numeric(dataset['Gross national income (GNI) per capita'])

fig_hdi = px.scatter(dataset, x="Gross national income (GNI) per capita", y="Human Development Index (HDI) ", trendline="lowess")
fig_tons = px.scatter(dataset, x="Gross national income (GNI) per capita", y="Tons Per Capita", trendline="lowess")

fig_hdi.show()
fig_tons.show()

In [25]:
filtered_dataset = dataset[dataset['HUMAN DEVELOPMENT'] != "OTHER "]
fig = px.box(filtered_dataset, x="HUMAN DEVELOPMENT", y="CO2 Emissions")
fig.update_xaxes(title="Human Development")
fig.update_yaxes(title="CO2 Emissions in Mtons")
fig.update_xaxes(categoryorder='array', categoryarray=[ 'LOW', 'MEDIUM ', 'HIGH', 'VERY HIGH '])
fig.show()